In [1]:
"""
European Options in a Black-Scholes World
----------------------------------------

Key ideas
─────────
• European option – exercisable only at a fixed maturity  T.
• Black-Scholes world – friction-free market where the spot price S_t
  follows:  dS/S = r dt + σ dW_t  (GBM with constant r, σ).
• No-arbitrage price – discounted risk-neutral expectation of payoff.
  ▸ Call  : C = e^{-rT} · E_Q[(S_T − K)^+]
  ▸ Put   : P = e^{-rT} · E_Q[(K − S_T)^+]
  Black–Scholes closed form gives these expectations analytically.
• Greeks (Δ, Γ, Θ, Vega, Rho) – partial derivatives of price w.r.t.
  the inputs, measuring risk sensitivities.

The code below:
  1. Sets parameters.
  2. Implements vectorised Black–Scholes price & delta.
  3. Runs a fast NumPy Monte-Carlo to sanity-check the price.
     (All 100 000 paths in ≈ 0.05 s on a laptop.)
"""

import numpy as np
from scipy.stats import norm
rng = np.random.default_rng(seed=1)

# ----------------------------------------------------------------------
# 1. Contract & market parameters
# ----------------------------------------------------------------------
S0     = 100      # spot price
K      = 105      # strike
r      = 0.03     # risk-free rate (continuously compounded)
sigma  = 0.25     # volatility (annualised)
T      = 0.75     # maturity in years
Nmc    = 100_000  # Monte-Carlo paths

# ----------------------------------------------------------------------
# 2. Black–Scholes closed-form price, delta, and greeks
# ----------------------------------------------------------------------
def bs_price_delta(S, K, r, sigma, T, call=True):
    """Return price and delta for a European option."""
    if T == 0:                        # avoid divide-by-zero at expiry
        payoff = np.maximum(S - K, 0) if call else np.maximum(K - S, 0)
        delta  = (S > K).astype(float) if call else -(S < K).astype(float)
        return payoff, delta

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if call:
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        delta = norm.cdf(d1)
    else:  # put
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        delta = norm.cdf(d1) - 1
    return price, delta

C_bs, Delta_bs = bs_price_delta(S0, K, r, sigma, T, call=True)
P_bs, _        = bs_price_delta(S0, K, r, sigma, T, call=False)

print(f"Black–Scholes call price : {C_bs: .4f}")
print(f"Black–Scholes put  price : {P_bs: .4f}")
print(f"Call delta               : {Delta_bs: .4f}")

# ----------------------------------------------------------------------
# 3. Monte-Carlo verification (vectorised, lightning fast)
# ----------------------------------------------------------------------
# simulate S_T under risk-neutral measure
Z   = rng.standard_normal(Nmc)
ST  = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
call_payoff = np.maximum(ST - K, 0.0)
mc_price    = np.exp(-r * T) * call_payoff.mean()
se_price    = np.exp(-r * T) * call_payoff.std(ddof=1) / np.sqrt(Nmc)

print(f"MC price ± 1.96·SE      : {mc_price: .4f}  ±  {1.96*se_price: .4f}")


Black–Scholes call price :  7.4675
Black–Scholes put  price :  10.1313
Call delta               :  0.4947
MC price ± 1.96·SE      :  7.3820  ±   0.0836


In [ ]:
#Black-Scholes says: plug today’s stock price, volatility, rate, strike,
#and time to expiry into one formula—that number is the European option’s fair price.

In [ ]:
#Call option: a contract that lets you buy the stock at the fixed strike price on the expiry date, if you want.

#Put option: a contract that lets you sell the stock at the fixed strike price on the expiry date, if you want.